In [4]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import cv2
from PIL import Image
import os
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Dataset, random_split
import torchvision.transforms as transforms
from torchinfo import summary
import torchvision.models as models

In [5]:
file = pd.read_csv('./data/encoded_data_v2.csv')
file.tail(1)

C:\Users\kdp\AppData\Local\Temp\ipykernel_2088\3315460536.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  file = pd.read_csv('./data/encoded_data_v2.csv')


,Unnamed: 0,image_name,comment_number,comment,0,1,2,3,4,5,...,68,69,70,71,72,73,74,75,76,77
158914,158914,998845445.jpg,4,A man on a moored blue and white boat with hil...,0,2,0,0,6294,11,...,1,1,1,1,1,1,1,1,1,1


In [54]:
data = []
for k in range(file.shape[0]):
    sample = []
    img_dir = './data/flickr30k_images/'
    img_file = img_dir + file.iloc[k][1] 
    sample.append(img_file)

    capt = []
    for idx in range(78):
        capt.append(file.iloc[k][idx+4])
        # print(file.iloc[k][idx+4])

    sample.append(capt)
    data.append(sample)

KeyboardInterrupt: 

In [35]:
for k in range(2):
    print(data[k])

['./data/flickr30k_images/1000092795.jpg', [37, 10, 288, 0, 2075, 81, 147, 0, 0, 121, 0, 276, 0, 0, 0, 438, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]
['./data/flickr30k_images/1000092795.jpg', [37, 10, 7, 662, 0, 33, 55, 273, 1379, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]


In [36]:
# 이미지 전처리
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

In [37]:
# 이미지와 캡션 로드  (예시)
# image = Image.open("example.jpg")
# image = transform(image).unsqueeze(0)
# caption = torch.LongTensor([1, 2, 3, 4, 5])  # 예시 캡션
# caption.shape

In [50]:
# 이미지로드
image = Image.open(data[0][0])
image = transform(image).unsqueeze(0)
image.shape

torch.Size([1, 3, 224, 224])

In [39]:
# 캡션 로드
caption = data[0][1]
caption = torch.LongTensor(caption)
caption.shape

torch.Size([78])

In [48]:
class ImageCaptioningModel(nn.Module):
    def __init__(self, embed_size, hidden_size, vocab_size, num_layers=1):
        super(ImageCaptioningModel, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.cnn = models.resnet50()
        self.cnn.fc = nn.Linear(self.cnn.fc.in_features, embed_size)
        self.rnn = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)
    
    def forward(self, images, captions):
        features = self.cnn(images)
        embeddings = self.embed(captions)

        print(f"feature : {features.shape} embed : {embeddings.shape}")

        # embeddings = torch.cat((features.unsqueeze(1), embeddings), 1)
        embeddings = torch.cat((features, embeddings), 1)
        out, _ = self.rnn(embeddings)
        out = self.fc(out)
        return out

In [53]:
# 모델 초기화
embed_size = 256
hidden_size = 512
vocab_size = 21652  # 단어 사전 크기
model = ImageCaptioningModel(embed_size, hidden_size, vocab_size)

# 추론 실행
output = model(image, caption.unsqueeze)


TypeError: embedding(): argument 'indices' (position 2) must be Tensor, not builtin_function_or_method

<hr>

<hr>

<hr>

In [22]:
use_df = pd.DataFrame(data)
use_df.head()
use_df.to_csv('img+encod.csv')

In [7]:
for k in range(3):
    print(data[k])
    print(len(data[k][1]))
    print()

['./data/flickr30k_images/1000092795.jpg', [37, 10, 288, 0, 2075, 81, 147, 0, 0, 121, 0, 276, 0, 0, 0, 438, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]
78

['./data/flickr30k_images/1000092795.jpg', [37, 10, 7, 662, 0, 33, 55, 273, 1379, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]
78

['./data/flickr30k_images/1000092795.jpg', [37, 18, 0, 28, 209, 0, 16, 0, 0, 438, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]
78



In [8]:
# 사용자 정의 데이터셋 정의
class CustomDataset(Dataset):
    def __init__(self, data, transform):
        self.data = data
        self.transform = transform
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        # 이미지와 캡션을 로드하고 전처리하여 반환
        image_path, caption = self.data[idx]
        image = Image.open(image_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        caption = torch.LongTensor(caption)
        return image, caption

In [9]:
# 데이터셋 인스턴스 생성
transform = transforms.Compose([
    transforms.Resize((150,150)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])
dataset = CustomDataset(data, transform=transform)
print(len(dataset))

158915


In [10]:
for img, capt in dataset:
    print(img)
    print(type(img))
    print(capt)
    print(type(capt))
    break

tensor([[[-2.0665, -2.0323, -2.0323,  ...,  0.8276,  1.2214,  1.4269],
         [-2.0323, -2.0323, -1.9809,  ...,  0.4851, -0.3712, -0.1486],
         [-1.9638, -2.0323, -2.0323,  ..., -0.7137, -0.5767, -0.3369],
         ...,
         [-0.7308, -0.7479, -0.4054,  ...,  0.1939,  0.1939,  0.1939],
         [-0.4568, -0.1828, -0.0801,  ..., -0.0287,  0.2624,  0.3138],
         [ 0.4851,  0.2282,  0.2624,  ...,  0.2796,  0.4166,  0.3481]],

        [[-1.9482, -1.9307, -1.8957,  ...,  1.6408,  1.9559,  2.1835],
         [-1.9132, -1.9307, -1.8431,  ...,  1.3256,  0.6254,  0.9580],
         [-1.7906, -1.8957, -1.8957,  ...,  0.2577,  0.4853,  0.7479],
         ...,
         [-0.3550, -0.3550,  0.0651,  ...,  0.9755,  0.9930,  0.9930],
         [ 0.0476,  0.2052,  0.4678,  ...,  0.9405,  1.1155,  1.0805],
         [ 1.0105,  1.0455,  0.8880,  ...,  1.0280,  1.0980,  1.0630]],

        [[-1.7173, -1.7173, -1.6999,  ...,  1.6465,  2.1694,  2.3611],
         [-1.7173, -1.7173, -1.6476,  ...,  1

In [11]:
# dataset에서 train, valid, test를 나누기 
seed_gen = torch.Generator().manual_seed(42)
tr, val, ts = 0.7,0.1,0.2
trainDS, validDS, testDS = random_split(dataset, [tr, val, ts], generator=seed_gen)
print(len(trainDS), len(validDS), len(testDS))

111241 15891 31783


In [12]:
# dataloader 생성
batch_size = 256
train_dl = DataLoader(trainDS, batch_size=batch_size, shuffle=True, drop_last = True)
valid_dl = DataLoader(validDS, batch_size=batch_size, shuffle=True, drop_last = True)
test_dl = DataLoader(testDS, batch_size = batch_size, shuffle=True, drop_last = True)
print(len(train_dl), len(valid_dl), len(test_dl))

434 62 124


In [13]:
for img, capt in train_dl:
    print(capt)
    print()
    break

tensor([[    0, 14123,     2,  ...,     1,     1,     1],
        [    0,   397,     0,  ...,     1,     1,     1],
        [    0,     9,     2,  ...,     1,     1,     1],
        ...,
        [    0,     2,     0,  ...,     1,     1,     1],
        [    0,     3,     0,  ...,     1,     1,     1],
        [   37,    18,   185,  ...,     1,     1,     1]])



In [14]:
# 1개의 배치 안에 있는 이미지 확인
from torchvision.utils import make_grid

def show_batch(dl):
    """Plot images grid of single batch"""
    for (images, labels) in dl:
        fig,ax = plt.subplots(figsize = (16,12))
        ax.set_xticks([])
        ax.set_yticks([])
        ax.imshow(make_grid(images,nrow=16).permute(1,2,0))
        break
        
# show_batch(train_dl)

### 클래스 생성

### Image Captioning with Pytorch
- 필요한 모델 : CNN & RNN 
- 인코딩용 : CNN => Resnet
- 디코딩용 : RNN => LSTM
- CNN에서 나온 결과물을 LSTM에 연결 

In [15]:
# 인코딩용 CNN 모델 생성 : RESNET18 (가중치O)

resnet = models.resnet18()
# 전결합층 변경
resnet.fc = nn.Linear(in_features = 512, out_features = 1)

# 모델의 합성곱층 가중치 고정 (완전 연결층은 학습시켜야함)
for name, param in resnet.named_parameters():
    param.requires_grad = False
for name, param in resnet.fc.named_parameters():
    param.requires_grad = True 

In [16]:
# 디코딩용 RNN 모델 생성 : LSTM

from torch.nn.utils.rnn import pack_padded_sequence
# 패딩된 시퀀스를 실제 데이터 길이에 맞게 패킹하여 효율적인 연산을 수행할 수 있게 해줌

class decoder(nn.Module):
    def __init__(self, embed_size, hidden_size, vocab_size, num_layers = 1):
        super(decoder, self).__init__()
        self.embed = nn.Embedding(num_embeddings=vocab_size, embedding_dim = embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, vocab_size)
    
    def forward(self, features, captions, lengths):
        embedding = self.embed(captions)
        print(f"embedding : {embedding.shape}")
        print(f"features : {features.shape}")

        print(f"features.unsqueeze(1) : {features.unsqueeze(1).shape}")
        embedding = torch.cat((features.unsqueeze(1), embedding),1)
        
        packed = pack_padded_sequence(embedding, lengths, batch_first = True)
        hiddens, _ = self.lstm(packed)
        outputs = self.linear(hiddens[0])
        return outputs

In [17]:
# 학습을 위한 하이퍼 파라미터 정의
embed_size = 256
hidden_size = 512
vocab_size = 21652  # 단어사전 크기
num_layers = 1
model = decoder(embed_size, hidden_size, 21653, num_layers)
# criterion = nn.CrossEntropyLoss()
criterion = nn.NLLLoss()  # 이 손실함수를 써야함
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [18]:
# for image, caption in train_dl:
#     print(caption[0])
#     a = np.array(caption[0])

#     print(len(np.nonzero(a)[0]))
#     num_list = []
#     for ind in range(len(image)):
#         a = np.array(caption[ind])
#         num_list.append(len(np.nonzero(a)[0]))
#     print(num_list)
#     break

# torch.Tensor(num_list), len(num_list)

### 학습, 검증, 테스트 함수 정의

In [19]:
import torchmetrics.functional as metrics

def training(dataloader):
    model.train()
    loss_list = []
    acc_list = []
    precision_list = []
    recall_list = []
    f1score_list = []

    for images, captions in dataloader:        
        num_list = []
        for ind in range(len(images)):
            a = np.array(captions[ind])
            num_list.append(len(np.nonzero(a)[0]))
    
        lengths = torch.Tensor(num_list)     #배치 사이즈가 아니고 0이 아닌 것의 개수 

        targets = pack_padded_sequence(captions, lengths, batch_first = True, enforce_sorted=False)[0]

        # 이미지 특성 추출
        features = resnet(images)
        features = features.view(features.size(0), -1)

        # 예측
        # print(f"features {features.shape}, captions {captions.shape}, lengths {lengths.shape}")

        outputs = model(features, captions, lengths)
        print(outputs)

        # 역전파
        loss = criterion(outputs, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


training(train_dl)

embedding : torch.Size([256, 78, 256])
features : torch.Size([256, 1])
features.unsqueeze(1) : torch.Size([256, 1, 1])


RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 1 but got size 256 for tensor number 1 in the list.

In [ ]:
def training(dataloader):
    model.train()
    loss_list = []
    acc_list = []
    precision_list = []
    recall_list = []
    f1score_list = []

    for images, captions in dataloader:        
        num_list = []
        for ind in range(len(image)):
            a = np.array(caption[ind])
            num_list.append(len(np.nonzero(a)[0]))
    
        lengths = torch.Tensor(num_list)     #배치 사이즈가 아니고 0이 아닌 것의 개수 
        
        targets = pack_padded_sequence(captions, lengths, batch_first = True, enforce_sorted=False)[0]

        # 이미지 특성 추출
        features = resnet(images)
        features = features.view(features.size(0), -1)

        # 예측
        outputs = model(features, captions, lengths)
        print(outputs)
        # 역전파
        loss = criterion(outputs, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # 정확도
        acc = metrics.accuracy(outputs, outputs, task = 'multiclass') 
        precision = metrics.precision(outputs, outputs, task = 'multiclass')
        recall = metrics.recall(outputs, outputs, task = 'multiclass')
        f1score = metrics.f1_score(outputs, outputs, task = 'multiclass')
        
        loss_list.append(loss)
        acc_list.append(acc)
        precision_list.append(precision)
        recall_list.append(recall)
        f1score_list.append(f1score)
        
    total_loss = sum(loss_list) / len(loss_list)
    total_acc = sum(acc_list) / len(acc_list)
    total_precision = sum(precision_list) / len(precision_list)
    total_recall = sum(recall_list)/len(recall_list)
    total_f1score = sum(f1score_list) / len(f1score_list)
    print(f"{total_loss} {total_acc} {total_precision} {total_recall} {total_f1score}")
    return total_loss, total_acc, total_precision, total_recall, total_f1score

training(train_dl)